<a href="https://colab.research.google.com/github/c-c-c-c/100knocks/blob/master/DeepDream_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip unistall keras
# !pip install keras==2.0.3

import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [2]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

# ここでは訓練しないため、訓練関連の演算は全て無効にする
K.set_learning_phase(0)

# InceptionV3ネットワークを畳み込みベースなしで構築
# このモデルは学習ずみのImageNetの重み付きで読み込まれる
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [3]:
# 層の名前を係数にマッピング
# 係数は最大化の対象となる損失値に、その層の活性化がどれくらい貢献するか
# 層の名前はInceptionV3にハードコードされている（model.summary()で確認できる）

layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [4]:
# model.summary()
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [5]:
# 層の名前を層のインスタンスにマッピング
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# 損失値を定義
loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
# 層の出力を取得
  activation = layer_dict[layer_name].output
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
# 層の特徴量のL2ノルムをLossに加算
  loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [6]:
# # Get the symbolic outputs of each "key" layer (we gave them unique names).
# layer_dict = dict([(layer.name, layer) for layer in model.layers])

# # Define the loss.
# loss = K.variable(0.)
# for layer_name in layer_contributions:
#     # Add the L2 norm of the features of a layer to the loss.
#     coeff = layer_contributions[layer_name]
#     activation = layer_dict[layer_name].output

#     # We avoid border artifacts by only involving non-border pixels in the loss.
#     scaling = K.prod(K.cast(K.shape(activation), 'float32'))
#     loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [7]:

# This holds our generated image
dream = model.input

# Compute the gradients of the dream with regard to the loss.
grads = K.gradients(loss, dream)[0]

# Normalize gradients.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [8]:
# # 生成されたドリーム画像を保持するテンソル
# dream = model.input
# # ドリームの損失関数の勾配を計算
# import tensorflow as tf
# from tensorflow import GradientTape
# # grads = K.gradients(loss, dream)[0]
# '''
# RuntimeError: tf.gradients is not supported 
# when eager execution is enabled. Use tf.GradientTape instead.
# -> GradientTapeにしてもX
# '''

# # from tensorflow import GradientTape
# grads = GradientTape(loss, dream)
# '''
# ValueError: Attempt to convert a value (<tensorflow.python.eager.backprop.GradientTape 
# object at 0x7fae74a2bf98>) with an unsupported type (<class 'tensorflow.python.eager.
# backprop.GradientTape'>) to a Tensor.
# '''

# # import tensorflow as tf
# # grads = tf.Graph()
# # with grads.as_default():
# #     grads = tf.gradients(loss, dream)[0]

# '''
# TypeError: Cannot convert a symbolic Keras input/output to a numpy array. 
# This error may indicate that you're trying to pass a symbolic value to a NumPy call, 
# which is not supported. Or, you may be trying to pass Keras symbolic inputs/outputs 
# to a TF API that does not register dispatching, preventing Keras from automatically 
# converting the API call to a lambda layer in the Functional Model.
# '''

# # 勾配を正規化（重要）
# grads = grads / K.maximum(K.mean(K.abs(grads)), 1e-7)
# # 入力画像に基づいて損失と勾配の値を取得するKeras関数を設定
# outputs = [loss, grads]
# fetch_loss_and_grads = K.function([dream], outputs)

# def eval_loss_andgrads(x):
#   outs = fetch_loss_and_grads([x])
#   loss_value = outs[0]
#   grad_values = outs[1]
#   return loss_value, grad_values

# def gradient_ascent(x, iterations, step, max_loss=None):
#   for i in range(iterations):
#     loss_value, grad_values = eval+loss_and_grads(x)
#     if max_loss is not None and loss_value >max_loss:
#       break
#     print('...loss value at', i, ':', loss_value)
#     x+= step * grad_values
#   return x

In [37]:
# !pip uninstall -y scipy
# !pip install scipy==0.15.1

# import scipy

!apt install freeimage

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package freeimage


In [42]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = '/content/hand.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
# for i in range(1, num_octave):
#   shape = tuple([int_dim / (octave_scale **i)) for dim in original_shape])
#   successive_shapes.append(shpae)
  
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
  print('Processing image shape', shape)
  img = resize_img(img, shape)
  img = gradient_ascent(img,
                        iterations=iterations,
                          step=step,
                          max_loss=max_loss)

  
  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
  same_size_original = resize_img(original_img, shape)
  lost_detail = same_size_original - upscaled_shrunk_original_img
  img += lost_detail
  shrunk_original_img = resize_img(original_img, shape)
  save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

Processing image shape (452, 612)
...Loss value at 0 : 1.4046388
...Loss value at 1 : 1.8369086
...Loss value at 2 : 2.5691853
...Loss value at 3 : 3.2257445
...Loss value at 4 : 3.9715176
...Loss value at 5 : 4.7218
...Loss value at 6 : 5.4354887
...Loss value at 7 : 6.165918
...Loss value at 8 : 6.8730354
...Loss value at 9 : 7.593799
...Loss value at 10 : 8.271179
...Loss value at 11 : 8.960426
...Loss value at 12 : 9.614954
Processing image shape (632, 857)
...Loss value at 0 : 3.395297
...Loss value at 1 : 4.7819963
...Loss value at 2 : 5.8856006
...Loss value at 3 : 6.8916492
...Loss value at 4 : 7.8377037
...Loss value at 5 : 8.712428
...Loss value at 6 : 9.546206
Processing image shape (886, 1200)
...Loss value at 0 : 3.3258421
...Loss value at 1 : 4.566557
...Loss value at 2 : 5.5755672
...Loss value at 3 : 6.5189867
...Loss value at 4 : 7.419509
...Loss value at 5 : 8.268315
...Loss value at 6 : 9.078313
...Loss value at 7 : 9.8482065


In [41]:
import scipy
from tensorflow.keras.preprocessing import image

def resize_img(img, size):
  img=np.copy(img)
  factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2] ,1)
  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  pil_img = deprocess_image(np.copy(img))

  import imageio
  imageio.imwrite('fname.jpg', pil_img)
#   scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = inception_v3.preprocess_input(img)
  return img

def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1, 2, 0))
  else:
    # 
    x = x.reshape((x.shape[1], x.shape[2], 3))
  x /= 2.
  x += 0.5
  x *= 255.
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [40]:
!apt install imageio 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package imageio
